In [2]:
import os
import numpy as np
import librosa
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.decomposition import PCA

In [8]:
current_directory = os.getcwd()


In [17]:
os.chdir(current_directory)


In [6]:
import pickle

# Cargar la lista desde el archivo .pkl
with open('canciones.pkl', 'rb') as file:
    canciones_cargadas = pickle.load(file)

# Ahora canciones_cargadas contiene la lista que guardaste anteriormente
print(len(canciones_cargadas))
print(canciones_cargadas[0])


5000
[-1.70336319e+02  1.07375282e+02  1.11163034e+01  3.55532799e+01
  2.31087089e+00  2.66392670e+01  3.40496850e+00  1.06783581e+01
 -5.40910959e+00  1.18548155e+01 -1.43089333e+01  2.85893273e+00
 -9.95297050e+00  3.84726691e+00 -9.87908649e+00  1.24272752e+00
 -1.14414740e+01  2.19873309e+00 -9.16021538e+00 -4.13439609e-02
  9.53358078e+01  3.52745361e+01  1.99946594e+01  1.44615955e+01
  1.25608215e+01  1.24151487e+01  1.14375238e+01  1.00703602e+01
  1.12153616e+01  1.12347870e+01  1.01768084e+01  9.06831264e+00
  8.83413601e+00  8.95058441e+00  8.27093124e+00  8.76001930e+00
  9.77761173e+00  9.67029572e+00  8.76695538e+00  1.00958214e+01
 -1.19900117e-08  1.49875146e-09  2.90383095e-09  0.00000000e+00
 -1.77976744e-09 -7.49375728e-10  1.31140754e-09  1.87343932e-10
 -8.43047743e-10 -2.34179925e-10  8.43047743e-10 -2.81015905e-10
 -5.62031810e-10  8.43047743e-10  9.36719660e-11 -1.17089960e-09
 -3.74687864e-10 -4.21523871e-10  0.00000000e+00  4.68359830e-11
  1.02082052e+01  3.

In [4]:
import numpy as np

# Cargar el archivo features.pkl con allow_pickle=True
#caractersiticas_Existentes = np.load('features.pkl', allow_pickle=True)
with open('id_caracteristicas.txt', 'r', encoding='utf-8') as file:
    nombres_id = file.readlines()

print(len(nombres_id))

5000


In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#NORMALIZAR data, para mejor busquedas 
data_normalized = scaler.fit_transform(canciones_cargadas)
#porque???? en diversos parpers lo dice



In [8]:
import os
import numpy as np
import librosa
from sklearn.preprocessing import StandardScaler


In [9]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from queue import PriorityQueue

def knn_search_priority_queue(query, k):
    # Hasta el de mejor resultados
    similarities = cosine_similarity(query.reshape(1, -1), data_normalized).flatten()
    
    # cola de prioridad
    priority_queue = PriorityQueue()
    for i, sim in enumerate(similarities):
        priority_queue.put((-sim, nombres_id[i]))  # por ser una cola para sacar los mejores(mas abajo ) desde abajo los metemos
    

    neighbors = []
    for _ in range(k):
        sim, neighbor = priority_queue.get()
        neighbors.append((neighbor, -sim))
    
    return neighbors


In [10]:

k = 5
nueva_cancion_features =  data_normalized[3481]#pasarle una cancion normalizada que feo yo queria pasarle una nueva
#nueva_cancion_features =  extract_features("Don Omar, Lucenzo - Danza Kuduro.mp3")

In [11]:
#Ya no es neceesario nomrlaizar
"""
query_normalized = scaler.transform(nueva_cancion_features.reshape(1, -1))
knn_result_priority_queue = knn_search_priority_queue((query_normalized), k)
"""
knn_result_priority_queue = knn_search_priority_queue((nueva_cancion_features), k)
print("Vecinos más cercanos (KNN con cola de prioridad): ")

for i, data in knn_result_priority_queue:
    print(i,":" ,data)


Vecinos más cercanos (KNN con cola de prioridad): 
1KyA5lXW7cAgMLwxRdVAu3.mp3
 : 0.9999999999999999
0IXpUl1fn2QZcBavfuq0H4.mp3
 : 0.8844898777490808
0rZTJmLQx7IquBctp98d0z.mp3
 : 0.869449540257319
0TcOuAT1fL2RA1YUgTrBvK.mp3
 : 0.8668417267510193
0fsj3HxzBHev6EJ7OLFE3r.mp3
 : 0.8637274229837487


In [35]:
import numpy as np

def knn_range_search(query, radio):

    vecinos_en_radio = []

    for i, sample in enumerate(data_normalized):
        distancia = euclidean_distances(query.reshape(1, -1), sample.reshape(1, -1))[0][0]
        if distancia is not None and distancia <= radio:
            vecinos_en_radio.append((nombres_id[i], distancia))
    
    vecinos_en_radio = sorted(vecinos_en_radio, key=lambda x: x[1])

    return vecinos_en_radio


In [45]:
r=10
rango= knn_range_search((nueva_cancion_features), r)
print(f"Vecinos más cercanos (range search):")
for i, data in rango:
    print(i, data)



Vecinos más cercanos (range search):
1KyA5lXW7cAgMLwxRdVAu3.mp3
 0.0
0IXpUl1fn2QZcBavfuq0H4.mp3
 9.17119148670579
0fsj3HxzBHev6EJ7OLFE3r.mp3
 9.657342771391605
0rZTJmLQx7IquBctp98d0z.mp3
 9.949004161781051


In [48]:
def knn_sequential(query, k):
    distances = []
    for sample in data_normalized:
        distance = euclidean_distances(query.reshape(1, -1), sample.reshape(1, -1))[0][0]
        distances.append(distance)
    
    indices = np.argsort(distances)[:k]#indices de manera ascendendete
    
    neighbors = []
    for index in indices:
        neighbors.append((nombres_id[index], distances[index]))
    
    return neighbors


In [49]:
k=5


knn_result_knn_sequential= knn_sequential((nueva_cancion_features), k)
print(f"Vecinos más cercanos (KNN Secuencial):")
for i, data in knn_result_knn_sequential:
    print(i, data)


Vecinos más cercanos (KNN Secuencial):
1KyA5lXW7cAgMLwxRdVAu3.mp3 0.0
0IXpUl1fn2QZcBavfuq0H4.mp3 9.17119148670579
0fsj3HxzBHev6EJ7OLFE3r.mp3 9.657342771391605
0rZTJmLQx7IquBctp98d0z.mp3 9.949004161781051
0TcOuAT1fL2RA1YUgTrBvK.mp3 10.169012871172837


In [52]:
import pandas as pd
import numpy as np
import  faiss

In [55]:
def train_index_HNSWFlat(data):
    # Inicializa el índice de FAISS con HNSW (Hierarchical Navigable Small World)
    M = 32  # Número de vecinos en la lista de entrada
    efConstruction = 100  # explorar al construir canciones mientras mayor es mejor la precision y mas demora
    dimension = 1000  # Dimensión de tus características
    quantizer = faiss.IndexHNSWFlat(dimension, M, faiss.METRIC_L2)#distancia eulediana
    quantizer.hnsw.efConstruction = efConstruction
    index = quantizer
    index.add(data)
    return index
"""
Sí, eso es correcto. En el contexto del índice HNSW con el parámetro M establecido en 32, cada punto (en este caso, cada canción) 
está directamente conectado a un máximo de 32 puntos en su lista de entrada. Estos son sus "vecinos cercanos" en el espacio de características.
""""""En otras palabras, cada canción en tu conjunto de datos tiene conexiones directas con hasta M 
canciones más cercanas según la métrica de distancia especificada (en este caso, la distancia L2)."""
def knn_faiss_HNSWFlat(indice, query_object, k):
    distances, indices = indice.search(np.expand_dims(query_object, axis=0), k)
    resultados = []
    for i in range(k):
        distancia = distances[0][i]
        etiqueta = nombres_id[indices[0][i]]
        resultados.append((distancia, etiqueta))
    return resultados

# Entrenamiento con IndexHNSWFlat
index_train = train_index_HNSWFlat(data_normalized)



In [56]:
query_example = nueva_cancion_features
k_result = knn_faiss_HNSWFlat(index_train, query_example, 5)
print(len(k_result))

print("Resultados de búsqueda KNN con FAISS (IndexHNSWFlat):")
for a, i in k_result:
    print(f"Distancia: {a}, Etiqueta: {i}")

5
Resultados de búsqueda KNN con FAISS (IndexHNSWFlat):
Distancia: 0.0, Etiqueta: 1KyA5lXW7cAgMLwxRdVAu3.mp3

Distancia: 84.11074829101562, Etiqueta: 0IXpUl1fn2QZcBavfuq0H4.mp3

Distancia: 93.26427459716797, Etiqueta: 0fsj3HxzBHev6EJ7OLFE3r.mp3

Distancia: 98.98268127441406, Etiqueta: 0rZTJmLQx7IquBctp98d0z.mp3

Distancia: 103.40882873535156, Etiqueta: 0TcOuAT1fL2RA1YUgTrBvK.mp3

